# 도봉구 코로나 시각화

In [6]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

import plotly as py
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import cufflinks as cf
cf.go_offline(connected=True)
from plotly.subplots import make_subplots
from plotly.offline import iplot

import geopandas as gpd
import geojson as gj
from geopandas.tools import sjoin

import shapely
from shapely.geometry import Point, Polygon, LineString

# 맵박스
import pydeck as pdk
from mapboxgl.utils import create_color_stops, create_numeric_stops

# 맵박스 API KEY 
MAPBOX_API_KEY = "pk.eyJ1IjoiamNsYXJhODExIiwiYSI6ImNrZzF4bWNhdTBpNnEydG54dGpxNDEwajAifQ.XWxOKQ-2HqFBVBYa-XoS-g"

from tqdm.notebook import tqdm
import time

pio.templates.default = "gridon"

# color pallette
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 
DEFAULT_PLOTLY_COLORS=['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                       'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
                       'rgb(148, 103, 189)', 'rgb(140, 86, 75)',
                       'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
                       'rgb(188, 189, 34)', 'rgb(23, 190, 207)']

# 기본 폰트 설정
layout_font = {'font':dict(size=18,color='#60606e',family='Franklin Gothic' )}

In [7]:
import shapely # Shapely 형태의 데이터를 받아 내부 좌표들을 List안에 반환합니다. 
def line_string_to_coordinates(line_string): 
    if isinstance(line_string, shapely.geometry.linestring.LineString): 
        lon, lat = line_string.xy 
        return [[x, y] for x, y in zip(lon, lat)] 
    elif isinstance(line_string, shapely.geometry.multilinestring.MultiLineString): 
        ret = [] 
        for i in range(len(line_string)): 
            lon, lat = line_string[i].xy 
            for x, y in zip(lon, lat): 
                ret.append([x, y])
        return ret 

def multipolygon_to_coordinates(x): 
    lon, lat = x[0].exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

def polygon_to_coordinates(x): 
    lon, lat = x.exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

def to_coordinates(x): 
    try: 
        lon, lat = x[0].exterior.xy 
    except:
        lon, lat = x.exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

In [8]:
# create our custom_dark theme from the plotly_dark template
pio.templates["custom_dark"] = pio.templates["plotly_dark"]

# set the paper_bgcolor and the plot_bgcolor to a new color
pio.templates["custom_dark"]['layout']['paper_bgcolor'] = '#222831'
pio.templates["custom_dark"]['layout']['plot_bgcolor'] = '#222831'

# you may also want to change gridline colors if you are modifying background
pio.templates['custom_dark']['layout']['yaxis']['gridcolor'] = '#d3e0ea'
pio.templates['custom_dark']['layout']['xaxis']['gridcolor'] = '#d3e0ea'

pio.templates.default = "custom_dark"

In [9]:
color_sequence=['#5D0000',
                '#930000',
                '#A50400',
                '#B71600',
                '#C92800',
                '#ED4C00',
                '#FF5E00',
                '#FF7012',
                '#FF9436',
                '#FFB85A',
                '#FFDC7E',
                '#FFEE90',
                '#FFFFA2',
                '#FFFFB4',
                '#FFFFC6',
               ]

In [10]:
root = Path('D:/2021데이터분석업무\결과물\기획/[일반] 4. 코로나 확진자 분석_20210309_남상원/Data')
# 도봉구 확진자
df = pd.read_csv(root/'C_dobong.csv', encoding='cp949', index_col = 'Unnamed: 0')
# 서울시 확진자
df2 = pd.read_csv(root/'C_seoul.csv', encoding='euc-kr')

## 1. 자료구조 확인

In [11]:
df.tail()

,확진번호,확진일,생년월일,성별,주소,동별,감염경로,lon,lat,나이,연령대
955,5,2020-03-27,630614,여,도봉구 우이천로 154 B03,창3동,해외입국자,127.038792,37.634423,59,5
956,4,2020-03-24,990204,여,"도봉구 노해로70길 12, 7동 809호(창동, 동아아파트)",창4동,해외입국자,127.050722,37.652111,23,2
957,3,2020-03-22,971016,남,"도봉구 마들로 657, 101동 701호(방학동, 이에스에이아파트)",방학1동,해외입국자,127.040683,37.664135,25,2
958,2,2020-03-14,590105,여,도봉구 창5동 대동아파트101-303,창5동,타구 확진자 접촉 추정,127.044072,37.661719,63,6
959,1,2020-03-02,821024,남,"도봉구 도봉로106길 22, 202동 1503호 (창동, 북한산현대홈시티)",창2동,타구 확진자 접촉 추정,127.034045,37.644998,40,4


---
## 2. 통계
### 2-1) 성별

In [12]:
sex =  df.groupby(['성별'], as_index = False).count()
sex.columns.values[1] = 'count'
sex = sex[['성별','count']]
fig = px.pie(sex, values = 'count', names = '성별', color = '성별',
            color_discrete_map={'남':'3f72af','여':'e23e57'},
            hole=.2,
            width = 480, height = 485)

fig.update_traces(textposition='inside', textinfo='percent+label')

fig.update_layout(font=dict(
                      family="Franklin Gothic",
                      size=20),
                  uniformtext_minsize=18, uniformtext_mode='hide')

# title
fig.update_layout(
    title_text="<b>성별 통계<b>",
    title_x=0.5,
    titlefont=dict(color="white",
                   size = 30),
)

fig.update_layout(
    title={
        "yref": "paper",
        "y" : 1,
        "yanchor" : "bottom"   
  }
)

fig.update_layout(
    legend=dict(x=1,y=0.5,font=dict(size=25)),
    margin=dict(l=20, r=20, t=55, b=0)
)

fig.update_layout(width = 480, height = 485)

fig.show()
#py.io.write_html(fig, file = f"{root}/3. 성별 통계.html",
#                 auto_play =True )

### 2-2) 연령대별

In [13]:
age =  df.groupby(['연령대']).count()
age.columns.values[1] = 'count'
age = age[['count']]
age = age.rename(index = {0:"10대 미만",
                          1:"10대",
                          2:"20대",
                          3:"30대",
                          4:"40대",
                          5:"50대",
                          6:"60대",
                          7:"70대",
                          8:"80대",
                          9:"90대"})

fig = px.pie(age, values = 'count', names = age.index, 
             color_discrete_sequence = px.colors.sequential.RdBu,
             hole = .2,
             width = 480, height = 485)

fig.update_traces(textposition='inside',
                  textinfo='percent+label',
                  textfont_size = 16)

fig.update_layout(font=dict(
                      family="Franklin Gothic",
                      size=14),
                  uniformtext_minsize=15, 
                  uniformtext_mode='hide')


# Add figure title
fig.update_layout(
    title_text="<b>연령대별 통계<b>",
    title_x=0.5,
    titlefont=dict(color="white",
                   size = 30),
)

fig.update_layout(
    title={
        "yref": "paper",
        "y" : 1,
        "yanchor" : "bottom"   
  }
)

fig.update_layout(
    legend=dict(x=1,y=0.5,font=dict(size=15)),
    margin=dict(l=20, r=20, t=55, b=0)
)

fig.update_layout(width = 480, height = 485)

fig.show()
fig.write_html(f"{root}/7. 연령대별 통계.html")

### 2-3) 감염경로별

In [14]:
route = df.groupby(['감염경로']).count()
route.columns.values[1] = 'count'
route = route[['count']]

fig = px.pie(route, values = 'count', names = route.index, 
             color_discrete_sequence = px.colors.sequential.RdBu,
             hole = .2)

fig.update_traces(textposition='inside',
                  textinfo='percent+label',
                  textfont_size = 16)

fig.update_layout(title_text = "<b>감염경로별 통계<b>", title_x=0.44,
                  font=dict(
                      family="Franklin Gothic",
                      size=14),
                  uniformtext_minsize=15, 
                  uniformtext_mode='hide')

fig.show()
#fig.write_html(f"{root}/3. 감염경로별 통계.html")

In [15]:
a = route.loc['도봉구'] + route.loc['도봉구(가족)']
b = route.loc['타구 확진자 가족'] + route.loc['타구 확진자 접촉 추정'] + route.loc['해외입국자'] + route.loc['확진자 접촉 추정'] 
c = route.loc['불명']

route_df = pd.DataFrame(data=[a,b], index= ['관내','관외'], columns=['count'])
pd.DataFrame(route_df)

fig = px.pie(route_df, values = 'count', names = route_df.index, color = route_df.index,
             color_discrete_map={'관내':'#ff2e63','관외':'#eaeaea'},
             hole = .2)

fig.update_traces(textposition='inside',
                  textinfo='percent+label',
                  textfont_size = 16)

fig.update_layout(font=dict(
                      family="Franklin Gothic",
                      size=18),
                  uniformtext_minsize=15, 
                  uniformtext_mode='hide')

# Add figure title
fig.update_layout(
    title_text="<b>감염경로별 통계<b>",
    title_x=0.5,
    titlefont=dict(color="white",
                   size = 30),
)

fig.update_layout(
    title={
        "yref": "paper",
        "y" : 1,
        "yanchor" : "bottom"   
  }
)

fig.update_layout(
    legend=dict(x=1,y=0.5,font=dict(size=20)),
    margin=dict(l=20, r=20, t=55, b=0)
)

fig.update_layout(width = 480, height = 485)

fig.show()
#fig.write_html(f"{root}/4. 감염경로별 통계.html")

### 2-4) 동별

In [16]:
dong = df.groupby(['동별']).count()
dong.columns.values[1] = 'count'
dong = dong[['count']]
fig = px.pie(dong, values = 'count', names = dong.index, 
             color_discrete_sequence = color_sequence,
             hole = .2)

fig.update_traces(textposition='inside',
                  textinfo='percent+label',
                  textfont_size = 16)

# Add figure title
fig.update_layout(
    title_text="<b>행정동별 통계<b>",
    title_x=0.5,
    titlefont=dict(color="white",
                   size = 30),
)

fig.update_layout(font=dict(
                      family="Franklin Gothic",
                      size=14),
                  uniformtext_minsize=13, 
                  uniformtext_mode='hide')

fig.update_layout(width = 480, height = 485)

fig.update_layout(
    title={
        "yref": "paper",
        "y" : 1,
        "yanchor" : "bottom"   
  }
)

fig.update_layout(
    legend=dict(x=1,y=0.5,font=dict(size=15)),
    margin=dict(l=20, r=20, t=55, b=0)
)

fig.show()
#fig.write_html(f"{root}/2. 행정동별 통계.html")

---
## 3. 시계열

### 3-1) 도봉구 확진자 추이 (6번화면)

In [17]:
time_d = df.groupby(['확진일']).count()
time_d.columns.values[1] = 'count'
time_d = time_d[['count']]

In [18]:
pio.templates.default = "custom_dark"

fig = go.Figure(data=[
        go.Bar(name='도봉구 확진자', x = time_d.index, y = time_d['count'],
               marker_color='#fed049',
               marker_line_color='#fed049', marker_line_width= 0.8),]
               )

labels = ['1차 대유행', '2차 대유행', '다나병원', '3차 대유행', ]
d_labels = ['2020-03-24', '2020-08-15','2020-09-24','2020-12-25']

for i in range(0,4) :
    fig.add_annotation(
            x = d_labels[i],
            y = time_d.loc[d_labels[i]]['count'],
            xref="x",
            yref="y",
            text= labels[i],
            showarrow=True,
            font=dict(
                family="Franklin Gothic",
                size=16,
                color="#f5f5f5"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#d8ebe4",
            ax=20,
            ay=-30,
            bordercolor="#d8ebe4",
            borderwidth= 2,
            borderpad = 3,
            bgcolor="#007580",
            opacity= 1
            )
    
# Layout 
fig.update_layout(
    title_text="<b>도봉구 신규확진자 발생현황<b>",
    titlefont=dict(color="#c70039",
                   size = 30),
)
    
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode="x unified",
                 font=dict(
                      family="Franklin Gothic",
                      size=17),
                 width = 960, height = 485)

#fig.update_xaxes(showgrid = False)

iplot(fig)
#fig.write_html(f"{root}/6. 도봉구 확진자 추이.html")

### 3-2) 서울시 확진자 추이

In [19]:
time_s = df2.groupby(['확진일']).count()
time_s.columns.values[0] = 'count'
time_s = time_s[['count']]
#time_s.iplot(kind='bar',theme='space', title='확진자추이',color = 'red')

### 3-3) 서울시-도봉구 확진자 추이

In [20]:
time_sd = pd.merge(time_s,time_d, on = '확진일', how ='left')
time_sd.columns = ['서울시 확진자', '도봉구 확진자']
time_sd.fillna(0)
#time_sd.iplot(kind = 'bar', theme = 'space', title = '코로나 확진자 추이', color = ['red','blue'])

,서울시 확진자,도봉구 확진자
확진일,,
2020-01-24,1,0.0
2020-01-30,3,0.0
2020-01-31,3,0.0
2020-02-02,1,0.0
2020-02-05,2,0.0
...,...,...
2021-03-12,140,9.0
2021-03-13,112,5.0
2021-03-14,112,4.0


In [21]:
pio.templates.default = "plotly_dark"

fig = go.Figure(data=[
        go.Bar(name='서울시 확진자', x = time_sd.index, y = time_sd['서울시 확진자']*.2,
               marker_color='rgba(255, 0, 0, 0.5)',
               marker_line_color='#e23e57', marker_line_width= 0.8),
        go.Bar(name='도봉구 확진자', x = time_sd.index, y = time_sd['도봉구 확진자'],
               marker_color='rgba(44, 40, 255, 0.5)',
               marker_line_color='#2c14ff', marker_line_width= 0.8),
])

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
#fig.update_layout(xaxis_tickangle=-45)
#fig.update_traces(hovertemplate=None)
#fig.update_layout(hovermode="x unified")
#iplot(fig)

In [22]:
pio.templates.default = "plotly_dark"

fig = go.Figure(data=[
        go.Bar(name='도봉구 확진자', x = time_sd.index, y = time_sd['도봉구 확진자'],
               marker_color='orange',
               marker_line_color='orange', marker_line_width= 0.8),
])

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
#fig.update_layout(xaxis_tickangle=-45)
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode="x unified")
iplot(fig)
#fig.write_html(f"{root}/0. 코로나 확진자 추이.html")

In [23]:
pio.templates.default = "custom_dark"

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])



# Add traces
fig.add_trace(
    go.Bar(name='<b>도봉구 확진자<b>', x = time_sd.index, y = time_sd['도봉구 확진자'],
               marker_color='#f05454',
               marker_line_color='#f05454', marker_line_width= 1),
    secondary_y=False,
)
   
fig.add_trace(
    go.Scatter(name='<b>서울시 확진자<b>', x = time_sd.index, y = time_sd['서울시 확진자'],
               line=dict(color='#17b978', width= 1),
               marker_color='#17b978',line_shape='linear'),
    secondary_y=True,
)

# Set x-axis title
#fig.update_xaxes(title_text="<b>xaxis title<b>")

# Set y-axes titles
fig.update_yaxes(title_text="<b>도봉구 확진자</b>",  
                 titlefont=dict(color="#f05454",
                               size = 26),
                 secondary_y=False)

fig.update_yaxes(title_text="<b>서울시 확진자<b>", 
                 titlefont=dict(color="#17b978",
                               size = 15),
                 secondary_y=True, showgrid = False)

# Add figure title
fig.update_layout(
    title_text="<b>코로나 확진자 추이<b>",
    title_x=0.5,
    titlefont=dict(color="white",
                   size = 30),
)

#fig.update_layout(xaxis_tickangle=-45)
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode="x unified",
                  legend=dict(x=0,y=1,
                              traceorder="reversed",
                              font=dict(size=15)),
                  font=dict(
                      family="Franklin Gothic",
                      size=17),)

fig.update_layout(width = 480, height = 485)


iplot(fig)
#fig.write_html(f"{root}/1. 코로나 확진자 추이.html")

In [24]:
pio.templates.default = "custom_dark"

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add figure title
fig.update_layout(
    title_text="<b>코로나 확진자 추이<b>",
    titlefont=dict(color="#c70039",
                   size = 45),
)

# Add traces
fig.add_trace(
    go.Scatter(name='<b>도봉구 확진자<b>', x = time_sd.index, y = time_sd['도봉구 확진자'],
               line=dict(color='#ff2e63', width= 1.5),
               marker_color='#ff2e63',line_shape='spline'),
    secondary_y=False,
)
   
fig.add_trace(
    go.Scatter(name='<b>서울시 확진자<b>', x = time_sd.index, y = time_sd['서울시 확진자'],
               line=dict(color='#17b978', width= 1.5),
               marker_color='#17b978',line_shape='spline'),
    secondary_y=True,
)

# Set x-axis title
#fig.update_xaxes(title_text="<b>xaxis title<b>")

# Set y-axes titles
fig.update_yaxes(title_text="<b>도봉구 확진자</b>",  
                 titlefont=dict(color="#f05454",
                               size = 26),
                 secondary_y=False)

fig.update_yaxes(title_text="<b>서울시 확진자<b>", 
                 titlefont=dict(color="#17b978",
                               size = 15),
                 secondary_y=True, showgrid = False)

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
#fig.update_layout(xaxis_tickangle=-45)
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode="x unified",
                 font=dict(
                      family="Franklin Gothic",
                      size=17))
iplot(fig)
#fig.write_html(f"{root}/0. 코로나 확진자 추이.html")

### 4-1. 연령대별 통계 (7번화면)

In [25]:
age =  df.groupby(['연령대']).count()
age.columns.values[1] = 'count'
age = age[['count']]
age = age.rename(index = {0:"10대 미만",
                          1:"10대",
                          2:"20대",
                          3:"30대",
                          4:"40대",
                          5:"50대",
                          6:"60대",
                          7:"70대",
                          8:"80대",
                          9:"90대"})

## 4-2. 연령대별 시계열 통계 (8번화면)

In [26]:
df_age = df.copy()
df_age['연령대'].replace(
    {0:"10대",
    1:"10대",
    2:"20~30대",
    3:"20~30대",
    4:"40~50대",
    5:"40~50대",
    6:"60대 이상",
    7:"60대 이상",
    8:"60대 이상",
    9:"60대 이상"}, inplace=True)
df_age['month'] = pd.to_datetime(df_age.확진일).dt.to_period('1M')
df_age['count'] = 1
df_age_m = df_age.groupby(by=['month','연령대']).agg({'count':'sum'}).reset_index().sort_values(by=['month','연령대'])
df_age_m['month'] = df_age_m['month'].astype(str)
df_age_m = df_age_m.pivot(index='month', columns='연령대', values='count')
df_age_m = df_age_m.fillna(0)
df_age_m

연령대,10대,20~30대,40~50대,60대 이상
month,,,,
2020-03,0.0,2.0,3.0,1.0
2020-04,0.0,2.0,1.0,0.0
2020-05,3.0,5.0,1.0,1.0
2020-06,2.0,0.0,7.0,34.0
2020-07,0.0,0.0,2.0,0.0
2020-08,1.0,16.0,28.0,28.0
2020-09,1.0,7.0,23.0,38.0
2020-10,1.0,5.0,5.0,12.0
2020-11,2.0,15.0,22.0,51.0


In [27]:
pio.templates.default = "custom_dark"

labels = ['60대 이상', '40~50대', '20~30대', '10대' ]
colors = ['#ec4646', '#f8f1f1', '#11698e', '#16c79a']

line_size = [3, 2.5, 3, 2.5]

fig = go.Figure()

for i in range(0, 4):
    fig.add_trace(go.Scatter(x=df_age_m.index, y=df_age_m[labels[i]], 
                             mode='lines',
                             line_shape='spline',
                             name=labels[i],
                             line=dict(color=colors[i], width=line_size[i]),
                             connectgaps=True,
                            ))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
#fig.update_layout(xaxis_tickangle=-45)
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode="x unified",
                 font=dict(
                      family="Franklin Gothic",
                      size=17))

# Add figure title
fig.update_layout(
    title_text="<b>연령대별 확진자 추이<b>",
    title_x=0.5,
    titlefont=dict(color="white",
                   size = 30),
)

fig.update_layout(
    legend=dict(x=0,y=1,traceorder="reversed",
                font=dict(size=15)),
    width = 480, height = 485
)

iplot(fig)
#fig.write_html(f"{root}/8. 연령대별 시계열 통계.html")

---
# 3-25 작업
## 1) 사이즈 변경 (485 -> 292.5)

In [28]:
sex =  df.groupby(['성별'], as_index = False).count()
sex.columns.values[1] = 'count'
sex = sex[['성별','count']]
fig = px.pie(sex, values = 'count', names = '성별', color = '성별',
            color_discrete_map={'남':'3f72af','여':'e23e57'},
            hole=.2,
            width = 480, height = 485)

fig.update_traces(textposition='inside', textinfo='percent+label')

fig.update_layout(font=dict(
                      family="Franklin Gothic",
                      size=20),
                  uniformtext_minsize=18, uniformtext_mode='hide')

# title
fig.update_layout(
    title_text="<b>성별 통계<b>",
    title_x=0.5,
    titlefont=dict(color="white",
                   size = 25),
)

fig.update_layout(
    title={
        "yref": "container",
        "y" : 0.92,
        "yanchor" : "top"
  }
)

fig.update_layout(
    legend=dict(x=1,y=0.6,font=dict(size=22)),
    margin=dict(l=80, r=20, t=50, b=10)
)

fig.update_layout(width = 480, height = 485/2)

fig.show()
py.io.write_html(fig, file = f"{root}/3-1. 성별 통계.html")

In [29]:
a = route.loc['도봉구'] + route.loc['도봉구(가족)']
b = route.loc['타구 확진자 가족'] + route.loc['타구 확진자 접촉 추정'] + route.loc['해외입국자'] + route.loc['확진자 접촉 추정'] 
c = route.loc['불명']

route_df = pd.DataFrame(data=[a,b], index= ['관내','관외'], columns=['count'])
pd.DataFrame(route_df)

fig = px.pie(route_df, values = 'count', names = route_df.index, color = route_df.index,
             color_discrete_map={'관내':'#ff2e63','관외':'#eaeaea'},
             hole = .2)

fig.update_traces(textposition='inside',
                  textinfo='percent+label',
                  textfont_size = 16)

fig.update_layout(font=dict(
                      family="Franklin Gothic",
                      size=18),
                  uniformtext_minsize=15, 
                  uniformtext_mode='hide')

# Add figure title
fig.update_layout(
    title_text="<b>감염경로별 통계<b>",
    title_x=0.5,
    titlefont=dict(color="white",
                   size = 25),
)

fig.update_layout(
    title={
        "yref": "container",
        "y" : 0.92,
        "yanchor" : "top"
  }
)

fig.update_layout(
    legend=dict(x=1,y=0.6,font=dict(size=22)),
    margin=dict(l=80, r=20, t=50, b=10)
)

fig.update_layout(width = 480, height = 485/2)

fig.show()
fig.write_html(f"{root}/3-2. 감염경로별 통계.html")

In [30]:
dong_2 = pd.read_csv(root/'행정동별 확진자.csv', encoding='cp949')
dong_2.rename(columns = {'ADM_DR_NM' : '행정동'}, inplace = True)
dong_2

,행정동,확진자수,행정동별 인구,인구대비확진자
0,도봉2동,124,27785,0.004463
1,쌍문1동,59,21790,0.002708
2,쌍문2동,57,19246,0.002962
3,쌍문3동,80,16769,0.004771
4,쌍문4동,42,19262,0.002180
5,방학1동,60,28975,0.002071
6,방학2동,33,20282,0.001627
7,방학3동,75,27901,0.002688
8,창1동,58,25241,0.002298
9,창2동,96,29643,0.003239


In [31]:
dong_2

,행정동,확진자수,행정동별 인구,인구대비확진자
0,도봉2동,124,27785,0.004463
1,쌍문1동,59,21790,0.002708
2,쌍문2동,57,19246,0.002962
3,쌍문3동,80,16769,0.004771
4,쌍문4동,42,19262,0.002180
5,방학1동,60,28975,0.002071
6,방학2동,33,20282,0.001627
7,방학3동,75,27901,0.002688
8,창1동,58,25241,0.002298
9,창2동,96,29643,0.003239


## 2) 행정동별 거주인구 대비 확진자 비율

In [32]:
# 도봉구 행정동 확진자
dong_2 = pd.read_csv(root/'행정동별 확진자.csv', encoding='cp949')
dong_2.rename(columns = {'ADM_DR_NM' : '행정동'}, inplace = True)
dong_2 = dong_2.sort_values(by='인구대비확진자', ascending = False)

fig = px.pie(dong_2, values = '인구대비확진자', names = '행정동',
             color = '인구대비확진자',
             color_discrete_sequence = color_sequence,
             hole = .2)

fig.update_traces(textposition='inside',
                  textinfo='percent+label',
                  textfont_size = 16)

# Add figure title
fig.update_layout(
    title_text="<b>행정동별 확진자편차<b>",
    title_x=0.5,
    titlefont=dict(color="white",
                   size = 30),
)

fig.update_layout(font=dict(
                      family="Franklin Gothic",
                      size=14),
                  uniformtext_minsize=13, 
                  uniformtext_mode='hide')

fig.update_layout(width = 480, height = 485)

fig.update_layout(
    title={
        "yref": "paper",
        "y" : 1,
        "yanchor" : "bottom"   
  }
)

fig.update_layout(
    legend=dict(x=1,y=0.5,font=dict(size=15)),
    margin=dict(l=20, r=20, t=55, b=0)
)

fig.show()
#fig.write_html(f"{root}/4. 행정동별 확진자편차.html")

# 인터렉티브화 작업

In [33]:
# you may also want to change gridline colors if you are modifying background
pio.templates['custom_dark']['layout']['yaxis']['zerolinecolor'] = 'rgba(255, 255, 255, 0.9)'
pio.templates['custom_dark']['layout']['yaxis']['gridcolor'] = 'rgba(255, 255, 255, 0.3)'
pio.templates['custom_dark']['layout']['xaxis']['gridcolor'] = 'rgba(255, 255, 255, 0.3)'

pio.templates.default = "custom_dark"

In [34]:
test = pd.read_csv(root/'test.csv', encoding='cp949')
test1 = pd.read_csv(root/'연령대별 통계.csv', encoding='cp949')
result = pd.merge(test, test1, on='month', how = 'left')
result.rename(columns = {'count_y' : '확진자수','연령대_y' : '연령대'}, inplace = True)

In [35]:
# 인터렉티브 차트

fig = px.line(result, 
                x="month", 
                y="확진자수", 
                color="연령대",
                color_discrete_sequence=['#16c79a', '#11698e', '#f8f1f1', '#ec4646'],
                line_shape = 'linear',
                range_x=['2020-03','2021-03'],
                range_y=[0, 110],
                animation_frame= "month_upto",
                hover_data= {
                    'month_upto' : False, 
                    'month': False,
                    '연령대': True, 
                    '확진자수': True,
                    }             
             )

fig.update_layout(
    title_text="<b>연령대별 확진자 추이<b>",
    title_x=0.5,
    titlefont=dict(color="white",
                   size = 30),
)

fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode="x unified",
                 font=dict(
                      family="Franklin Gothic",
                      size=17))

fig.update_layout(
    legend=dict(x=0,y=1,traceorder="reversed",
                font=dict(size=15)),
    width = 480, height = 485
)    

py.io.write_html(fig, 
                 file = f"{root}/3. 성별 통계.html",
                 auto_play =True)

# 서울시-도봉구 접촉력 파악 (2021 / 04 / 09)

In [36]:
# 도봉구-서울시 접촉력 분석
df3 = pd.read_csv(root/"서울시 코로나19 확진자 현황.csv", encoding = 'euc-kr')
df3 = df3.rename({'지역':'거주구'}, axis = 'columns')
df3 = df3[~df3['거주구'].isin(['기타','타시도'])]

## 진행 프로세스

### 1. 최종시각화

- 1) 타구 > 도봉구
- 2) 도봉구 > 타구

### 2. 정제방법

- '접촉력' > '구' 추출

In [37]:
gu_list = ['종로','중구','용산','성동','광진','동대문','중랑','성북','강북',
           '도봉','노원','은평','서대문','마포','양천','강서','구로','금천','영등포','동작','관악','서초','강남','송파','강동']

In [38]:
for i in gu_list :
    df3.loc[df3.접촉력.str.contains(i), '접촉구'] = (i + '구') 

In [39]:
df3 = df3.dropna(subset=['접촉구'])
df3 = df3.replace({"접촉구":"중구구"},{"접촉구":"중구"})

In [40]:
df3.거주구.unique()

array(['서초구', '종로구', '중랑구', '동대문구', '송파구', '강남구', '서대문구', '영등포구', '성북구',
       '강동구', '노원구', '관악구', '마포구', '성동구', '구로구', '도봉구', '은평구', '중구',
       '동작구', '강서구', '용산구', '광진구', '강북구', '양천구', '강북구 ', '금천구'],
      dtype=object)

In [41]:
temp = df3[['접촉구','거주구','연번']].groupby(['접촉구','거주구'], as_index = False).count()
temp

,접촉구,거주구,연번
0,강남구,강남구,242
1,강남구,강동구,33
2,강남구,강북구,17
3,강남구,강서구,18
4,강남구,관악구,52
...,...,...,...
561,중랑구,용산구,3
562,중랑구,은평구,3
563,중랑구,종로구,7
564,중랑구,중구,4


In [42]:
# 1. 거주구 <- 접촉구
dobong = temp[temp['거주구']=='도봉구']
dobong = dobong[dobong.접촉구 != '도봉구']
dobong.rename(columns = {'연번': '확진자수'} , inplace = True)
dobong['정규화_확진자수'] = dobong['확진자수']/ dobong['확진자수'].max()
#dobong.to_csv(f"{root}/1번.csv", encoding = 'euc-kr')

In [43]:
dobong

,접촉구,거주구,확진자수,정규화_확진자수
9,강남구,도봉구,15,0.535714
34,강동구,도봉구,2,0.071429
54,강북구,도봉구,15,0.535714
72,강서구,도봉구,4,0.142857
96,관악구,도봉구,5,0.178571
140,구로구,도봉구,4,0.142857
182,노원구,도봉구,28,1.000000
220,동대문구,도봉구,15,0.535714
245,동작구,도봉구,3,0.107143
269,마포구,도봉구,3,0.107143


In [44]:
# 2. 거주구 -> 접촉구 
r_dobong = temp[temp['접촉구']=='도봉구']
r_dobong = r_dobong[r_dobong.거주구 != '도봉구']
r_dobong.rename(columns = {'연번': '확진자수'} , inplace = True)
r_dobong['정규화_확진자수'] = r_dobong['확진자수']/ r_dobong['확진자수'].max()
#r_dobong.to_csv(f"{root}/2번.csv", encoding = 'euc-kr')

In [45]:
r_dobong

,접촉구,거주구,확진자수,정규화_확진자수
195,도봉구,강남구,2,0.076923
196,도봉구,강북구,17,0.653846
197,도봉구,강서구,1,0.038462
198,도봉구,광진구,1,0.038462
199,도봉구,구로구,5,0.192308
200,도봉구,노원구,26,1.000000
202,도봉구,동대문구,1,0.038462
203,도봉구,동작구,1,0.038462
204,도봉구,마포구,4,0.153846
205,도봉구,서대문구,1,0.038462


## 서울시 행정구역 좌표 결합

In [46]:
crd = pd.read_csv(f"{root}/서울시 행정구역 좌표.csv", encoding = 'euc-kr')

In [47]:
temp = gpd.GeoDataFrame(crd, geometry = gpd.points_from_xy(crd.경도, crd.위도))

In [48]:
geo_crd = temp[['시군구명_한글', '경도', '위도', 'geometry']]
geo_crd.rename(columns = {'시군구명_한글': '접촉구'} , inplace = True)

In [49]:
temp = pd.merge(dobong, geo_crd, on = "접촉구", how = "left")
temp.rename(columns = {'경도': '접촉구_경도', '위도':'접촉구_위도', 'geometry':'접촉구_geometry'}, inplace = True)
geo_crd.rename(columns = {'접촉구': '거주구'} , inplace = True)
temp = pd.merge(temp, geo_crd, on = "거주구", how = "left")
temp.rename(columns = {'경도': '거주구_경도', '위도':'거주구_위도', 'geometry':'거주구_geometry'}, inplace = True)
temp.head()

,접촉구,거주구,확진자수,정규화_확진자수,접촉구_경도,접촉구_위도,접촉구_geometry,거주구_경도,거주구_위도,거주구_geometry
0,강남구,도봉구,15,0.535714,127.066409,37.495985,POINT (127.06641 37.49599),127.031767,37.665861,POINT (127.03177 37.66586)
1,강동구,도봉구,2,0.071429,127.146482,37.549208,POINT (127.14648 37.54921),127.031767,37.665861,POINT (127.03177 37.66586)
2,강북구,도봉구,15,0.535714,127.014716,37.646995,POINT (127.01472 37.64700),127.031767,37.665861,POINT (127.03177 37.66586)
3,강서구,도봉구,4,0.142857,126.822656,37.565762,POINT (126.82266 37.56576),127.031767,37.665861,POINT (127.03177 37.66586)
4,관악구,도봉구,5,0.178571,126.943807,37.465399,POINT (126.94381 37.46540),127.031767,37.665861,POINT (127.03177 37.66586)


## 서울시 폴리곤.geojson 로드

In [50]:
seoul = gpd.read_file(f'{root}/서울특별시_시군구.geojson')                     

In [51]:
seoul['coordinates'] = seoul['geometry'].apply(multipolygon_to_coordinates)
seoul.rename(columns = {'SIG_KOR_NM': '접촉구'} , inplace = True)
geo_seoul = seoul[['접촉구', 'coordinates']]
geo_seoul.head()

,접촉구,coordinates
0,광진구,"[[127.10166464162197, 37.57240065825002], [127..."
1,도봉구,"[[127.01508681387975, 37.64928834470861], [127..."
2,노원구,"[[127.11084941976728, 37.638408879281094], [12..."
3,성북구,"[[127.01059113253235, 37.58025377376853], [127..."
4,강북구,"[[127.00456669928406, 37.6850798098906], [127...."


In [52]:
db_to_other = pd.merge(geo_seoul, dobong, on = '접촉구', how = 'left' )
db_to_other = db_to_other.fillna(0)

In [53]:
db_to_other.head()

,접촉구,coordinates,거주구,확진자수,정규화_확진자수
0,광진구,"[[127.10166464162197, 37.57240065825002], [127...",0,0.0,0.000000
1,도봉구,"[[127.01508681387975, 37.64928834470861], [127...",0,0.0,0.000000
2,노원구,"[[127.11084941976728, 37.638408879281094], [12...",도봉구,28.0,1.000000
3,성북구,"[[127.01059113253235, 37.58025377376853], [127...",도봉구,28.0,1.000000
4,강북구,"[[127.00456669928406, 37.6850798098906], [127....",도봉구,15.0,0.535714


In [54]:
layer1 = pdk.Layer(
    'PolygonLayer',
    db_to_other,
    get_polygon = 'coordinates',
    get_fill_color = '[255*정규화_확진자수,0, 0, 400*정규화_확진자수]',
    pickable = True,
    auto_highlight = True,
    extruded = True,
    get_elevation = '정규화_확진자수',
    elevation_scale = 1500
)

layer2 = pdk.Layer(
    'ArcLayer',
    temp,
    get_source_position = '[거주구_경도, 거주구_위도]',
    get_target_position = '[접촉구_경도, 접촉구_위도]',
    get_width = '확진자수*0.5',
    get_source_color = '[255, 255, 120, 400*정규화_확진자수]',
    get_target_color = '[255, 0, 0, 400*정규화_확진자수]',      
    pickable = False,
    auto_highlight = True    
)


# Set the viewport location 
center = [126.99526, 37.5520]
view_state = pdk.ViewState( 
    longitude = center[0], 
    latitude = center[1], 
    zoom = 10,
    bearing = 5,
    pitch = 40, 
) 

# Render 
r = pdk.Deck(layers=[layer1, layer2],
             initial_view_state = view_state, 
             tooltip={"text": "구명 : {접촉구} \n 확진자수 : {확진자수}"},
             mapbox_key = MAPBOX_API_KEY
            )
r.to_html(f"{root}/geojson_layer.html", notebook_display=True)

TypeError: __init__() got an unexpected keyword argument 'mapbox_key'

In [ ]:
layer1 = pdk.Layer(
    'PolygonLayer',
    seoul,
    get_polygon = 'coordinates',
    get_fill_color = '[255,255,255]',
    pickable = True,
    auto_highlight = True
)

layer2 = pdk.Layer(
    'ArcLayer',
    temp,
    get_source_position = '[거주구_경도, 거주구_위도]',
    get_target_position = '[접촉구_경도, 접촉구_위도]',
    get_width = '확진자수*0.5',
    get_source_color = '[255, 255, 120]',
    get_target_color = '[255, 0, 0]',      
    pickable = True,
    auto_highlight = True    
)

# Set the viewport location 
center = [126.98526, 37.6020]
view_state = pdk.ViewState( 
    longitude = center[0], 
    latitude = center[1], 
    zoom = 10,
    bearing = -15,
    pitch = 50, 
) 

# Render 
r = pdk.Deck(layers=[layer1, layer2],
             initial_view_state = view_state, 
             tooltip={"text": "구명 : {접촉구} \n 확진자수 : {확진자수}"},
             mapbox_key = MAPBOX_API_KEY
            )
r.to_html()

In [ ]:
x_axis = list(range(data.shape[1] - 1))
y_axis = list(data.iloc[0,1:])

frames = []
for frame in range(1, 13):
    x_axis_frame = np.arange(frame)
    y_axis_frame = list(data.iloc[0,1:frame])
    curr_frame = go.Frame(data = [go.Scatter(x= x_axis_frame, y = y_axis_frame, mode ="lines",line_shape='spline')])
    frames.append(curr_frame)

fig = go.Figure(
    data = [go.Scatter(x = np.array([1]), y = np.array([0]), mode = "lines")], # list of traces
    layout = {
              "xaxis" : {"range": [0, data.shape[1]]},
              "yaxis" : {"range": [0, 40]}
             }, # dictionary of key value pairs
    frames = frames # list of frames or list of figure
    )

fig.update_layout(
    title_text="<b>연령대별 확진자 추이<b>",
    title_x=0.5,
    titlefont=dict(color="white",
                   size = 30),
)
       
fig.update_layout(
    legend=dict(x=0,y=1,traceorder="reversed",
                font=dict(size=15)),
    width = 480, height = 485
)    
    

In [ ]:
px.scatter(df_age_m,
            x= df_age_m.index, y= df_age_m['60대 이상'],
            animation_frame= df_age_m.index, 
            animation_group= df_age_m.index,
            size= df_age_m['60대 이상'], 
            color= df_age_m.index, 
            hover_name= df_age_m.index,
            log_x=True,
            size_max=55, 
            range_x=[100,100000], 
            range_y=[25,90])

In [ ]:
pio.templates.default = "custom_dark"

labels = ['60대 이상', '40~50대', '20~30대', '10대' ]
colors = ['#ec4646', '#f8f1f1', '#11698e', '#16c79a']

line_size = [3, 2.5, 3, 2.5]

fig = go.Figure()

for i in range(0, 4):
    fig.add_trace(px.Scatter(x=df_age_m.index, y=df_age_m[labels[i]], 
                             mode='lines',
                             line_shape='spline',
                             name=labels[i],
                             line=dict(color=colors[i], width=line_size[i]),
                             connectgaps=True,
                            ))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
#fig.update_layout(xaxis_tickangle=-45)
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode="x unified",
                 font=dict(
                      family="Franklin Gothic",
                      size=17))

# Add figure title
fig.update_layout(
    title_text="<b>연령대별 확진자 추이<b>",
    title_x=0.5,
    titlefont=dict(color="white",
                   size = 30),
)

fig.update_layout(
    legend=dict(x=0,y=1,traceorder="reversed",
                font=dict(size=15)),
    width = 480, height = 485
)

iplot(fig)
#fig.write_html(f"{root}/8. 연령대별 시계열 통계.html")

In [ ]:
df = px.data.gapminder()

fig = px.scatter(df, 
                 x="gdpPercap", y="lifeExp", 
                 animation_frame="year", 
                 animation_group="country",
                 size="pop", 
                 color="continent", 
                 hover_name="country",
                 log_x=True, 
                 size_max=55, 
                 range_x=[100,100000], 
                 range_y=[25,90])

# remove button
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            buttons = list([
                dict(visible= False)
                ])
        )
    ]
)


py.io.write_html(fig, 
                 file = f"{root}/3. 성별 통계.html",
                 auto_play =True)